In [ ]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [ ]:
# MPC import
import numpy as np
from LinearMPC_3_3.MPCVelControl import MPCVelControl
from PIControl.PIControl import PIControl
from src.rocket import Rocket
from src.vel_rocket_vis import RocketVis, plot_static_states_inputs

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

In [ ]:
Ts = 0.05
sim_time = 20
H = 5.0
x0 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 50, 50, 100])
pos_target = np.array([0, 0, 10.0])

print("Position Control Simulation")
print(f"Initial state: x={x0[9]:.1f}m, y={x0[10]:.1f}m, z={x0[11]:.1f}m")
print(f"Target position: x={pos_target[0]:.1f}m, y={pos_target[1]:.1f}m, z={pos_target[2]:.1f}m")

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
pos_controller = PIControl(pos_target)
mpc = MPCVelControl.new_controller(rocket, Ts, H)

t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, ref = rocket.simulate_control(
    mpc, sim_time, H, x0, pos_control=pos_controller, method="linear"
)

# Print final results
print(f"\nFinal Results:")
print(f"  Position:")
print(f"    x = {x_cl[9, -1]:.3f} m (target: {pos_target[0]:.1f} m)")
print(f"    y = {x_cl[10, -1]:.3f} m (target: {pos_target[1]:.1f} m)")
print(f"    z = {x_cl[11, -1]:.3f} m (target: {pos_target[2]:.1f} m)")
print(f"  Velocity:")
print(f"    vx = {x_cl[6, -1]:.3f} m/s")
print(f"    vy = {x_cl[7, -1]:.3f} m/s")
print(f"    vz = {x_cl[8, -1]:.3f} m/s")
print(f"  Orientation:")
print(f"    alpha = {np.rad2deg(x_cl[3, -1]):.2f}°")
print(f"    beta = {np.rad2deg(x_cl[4, -1]):.2f}°")
print(f"    gamma = {np.rad2deg(x_cl[5, -1]):.2f}°")

vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(t_cl[:-1], x_cl[:, :-1], u_cl, Ref=ref[:, :-1], T_ol=t_ol[..., :-1], X_ol=x_ol, U_ol=u_ol)

# Display static plots
plot_static_states_inputs(t_cl[:-1], x_cl[:, :-1], u_cl, ref[:, :-1])

**Closed-loop simulation with position control**

In [ ]:
# Closed-loop simulation: MPC + PI controller recalculate at each time step
x0_cl = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 50, 50, 100])
pos_target_cl = np.array([0, 0, 10.0])

print("CLOSED-LOOP Simulation (MPC + PI recalculated at each step)")
print(f"Initial position: x={x0_cl[9]:.1f}m, y={x0_cl[10]:.1f}m, z={x0_cl[11]:.1f}m")
print(f"Target position: x={pos_target_cl[0]:.1f}m, y={pos_target_cl[1]:.1f}m, z={pos_target_cl[2]:.1f}m")

rocket_cl = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
pos_controller_cl = PIControl(pos_target_cl)
mpc_cl = MPCVelControl.new_controller(rocket_cl, Ts, H)

# Closed-loop simulation
t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, ref_cl = rocket_cl.simulate_control(
    mpc_cl, sim_time, H, x0_cl, pos_control=pos_controller_cl, method="linear", sim="CLOSED_LOOP"
)

# Print results
print(f"\nClosed-Loop Results:")
print(f"  Position:")
print(f"    x = {x_cl[9, -1]:.3f} m (target: {pos_target_cl[0]:.1f} m)")
print(f"    y = {x_cl[10, -1]:.3f} m (target: {pos_target_cl[1]:.1f} m)")
print(f"    z = {x_cl[11, -1]:.3f} m (target: {pos_target_cl[2]:.1f} m)")
print(f"  Velocity:")
print(f"    vx = {x_cl[6, -1]:.3f} m/s")
print(f"    vy = {x_cl[7, -1]:.3f} m/s")
print(f"    vz = {x_cl[8, -1]:.3f} m/s")
print(f"  Orientation:")
print(f"    alpha = {np.rad2deg(x_cl[3, -1]):.2f}°")
print(f"    beta = {np.rad2deg(x_cl[4, -1]):.2f}°")
print(f"    gamma = {np.rad2deg(x_cl[5, -1]):.2f}°")

# Plot static graphs
n_points = min(len(t_cl), x_cl.shape[1], u_cl.shape[1])
plot_static_states_inputs(t_cl[:n_points], x_cl[:, :n_points], u_cl[:, :n_points], Ref=ref_cl[:, :n_points])

**Open-loop simulation with position control**

In [ ]:
# Open-loop simulation: Control computed once at t=0, no feedback
x0_ol = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 50, 50, 100])
pos_target_ol = np.array([0, 0, 10.0])

print("OPEN-LOOP Simulation (control computed once at t=0, no feedback)")
print(f"Initial position: x={x0_ol[9]:.1f}m, y={x0_ol[10]:.1f}m, z={x0_ol[11]:.1f}m")
print(f"Target position: x={pos_target_ol[0]:.1f}m, y={pos_target_ol[1]:.1f}m, z={pos_target_ol[2]:.1f}m")

rocket_ol = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
pos_controller_ol = PIControl(pos_target_ol)
mpc_ol = MPCVelControl.new_controller(rocket_ol, Ts, H)

# Get initial velocity reference from PI
vel_ref_0 = pos_controller_ol.get_u(x0_ol[9:12])
x_target_0 = np.zeros(12)
x_target_0[6:9] = vel_ref_0

# Compute optimal control sequence once at t=0
u_first, x_pred, u_seq, t_pred = mpc_ol.get_u(0.0, x0_ol, x_target=x_target_0)

# Extend control sequence to cover full simulation time
N_sim = int(sim_time / Ts)
N_horizon = u_seq.shape[1]

print(f"  MPC horizon: {H}s ({N_horizon} steps)")
print(f"  Simulation time: {sim_time}s ({N_sim} steps)")

# Create extended control sequence: use MPC prediction, then hold last value
U_extended = np.zeros((4, N_sim))
if N_horizon <= N_sim:
    U_extended[:, :N_horizon] = u_seq
    U_extended[:, N_horizon:] = u_seq[:, -1:].repeat(N_sim - N_horizon, axis=1)
else:
    U_extended = u_seq[:, :N_sim]

# Simulate with fixed control sequence over full sim_time (no MPC/PI replanning)
t_ol, x_ol, u_ol = rocket_ol.simulate(x0_ol, sim_time, U_extended, method="linear")

# Print results
print(f"\nOpen-Loop Results:")
print(f"  Position:")
print(f"    x = {x_ol[9, -1]:.3f} m (target: {pos_target_ol[0]:.1f} m)")
print(f"    y = {x_ol[10, -1]:.3f} m (target: {pos_target_ol[1]:.1f} m)")
print(f"    z = {x_ol[11, -1]:.3f} m (target: {pos_target_ol[2]:.1f} m)")
print(f"  Velocity:")
print(f"    vx = {x_ol[6, -1]:.3f} m/s")
print(f"    vy = {x_ol[7, -1]:.3f} m/s")
print(f"    vz = {x_ol[8, -1]:.3f} m/s")
print(f"  Orientation:")
print(f"    alpha = {np.rad2deg(x_ol[3, -1]):.2f}°")
print(f"    beta = {np.rad2deg(x_ol[4, -1]):.2f}°")
print(f"    gamma = {np.rad2deg(x_ol[5, -1]):.2f}°")

# Plot static graphs for open-loop
n_points_ol = min(len(t_ol), x_ol.shape[1], u_ol.shape[1])
plot_static_states_inputs(t_ol[:n_points_ol], x_ol[:, :n_points_ol], u_ol[:, :n_points_ol], Ref=x_target_0)